In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
from pprint import pprint as pp
import csv
from pathlib import Path
import seaborn as sns
from itertools import product
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import Pipeline 

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score, classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from sklearn.metrics import homogeneity_score, silhouette_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import MiniBatchKMeans, DBSCAN
import fasttext

import gensim
from gensim import corpora

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

In [4]:
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from gensim.models import Word2Vec
from gensim.models import FastText
import numpy as np
np.random.seed(42)

In [5]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_predict

In [6]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek

In [7]:
pd.set_option('display.max_columns', None)

### Retrieving the data

#### DF 7

In [8]:
#df7 = pd.read_csv('df7.csv')
df7 = pd.read_csv('df7.csv', index_col=0)

In [9]:
df7.head(2)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens,target,content
0,1077501,1296599.0,5000.0,5000.0,4975.0,36months,0.1065,162.87,B,B2,Unknown,10,RENT,24000.0,Verified,2011,False,credit_card,Computer,860,AZ,27.65,0.0,1985,735.0,739.0,1.0,500.0,222.0,3.0,0.0,13648.0,0.837,9.0,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.0,0.00,0.00,2015,171.62,0,2016,744.0,740.0,False,False,False,0.0,0.0,False,__label__1,__label__1 1077501 1296599.0 5000.0 5000.0 497...
1,1077430,1314167.0,2500.0,2500.0,2500.0,60months,0.1527,59.83,C,C4,Ryder,1,RENT,30000.0,Source Verified,2011,False,car,bike,309,GA,1.00,0.0,1999,740.0,744.0,5.0,500.0,222.0,3.0,0.0,1687.0,0.094,4.0,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.0,117.08,1.11,2013,119.66,0,2016,499.0,0.0,False,False,False,0.0,0.0,False,__label__2,__label__2 1077430 1314167.0 2500.0 2500.0 250...


# Models

In [10]:
# Initialize lists to store evaluation metrics for each fold
dataset_used = []
model_used = []
data_balancing_technique = []
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
conf_matrices = []

In [11]:
#combined_metrics = pd.DataFrame()

combined_metrics = pd.DataFrame(columns=['dataset', 'model', 'data balancing technique', 'fold', 'precision_1','precision_2','recall_1','recall_2','f1-score_1','f1-score_2','support_1','support_2','TP','FP','TN','FN'])

In [12]:
# Assuming 'df7' is your DataFrame with features and the target column
features = df7['content'].apply(lambda x: x.split(' ', 1)[1])  # Drop the target column to get the feature columns
target = df7['target'].apply(lambda x: int(x.split("__label__")[1]))  # Target column to predict

In [28]:
features = features.apply(preprocess)

In [13]:
stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [14]:
# Splitting the dataset into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [15]:
X_train = X_train.apply(preprocess)
X_test = X_test.apply(preprocess)

In [19]:
X_train.head(2)

40315    541691 6991370 122250 122250 122250 60months 0...
15335    751654 9511120 25250 25250 25250 60months 0179...
Name: content, dtype: object

In [20]:
sentences = [sentence.split() for sentence in X_train]
    
# Train the FastText model
fasttext_model = FastText(sentences, vector_size=100, window=20, min_count=2, workers=4, seed=42)

In [21]:
def vectorize(sentence):
        words = sentence.split()
        words_vecs = [fasttext_model.wv[word] for word in words if word in fasttext_model.wv]
        if len(words_vecs) == 0:
            return np.zeros(100)
        words_vecs = np.array(words_vecs)
        return words_vecs.mean(axis=0)

In [23]:
X_train = np.array([vectorize(sentence) for sentence in X_train])
X_test = np.array([vectorize(sentence) for sentence in X_test])

In [24]:
clf = LinearRegression()
clf.fit(X_train, y_train)

LinearRegression()

In [25]:
y_pred = clf.predict(X_test)
    
y_pred = np.round(y_pred)
    
y_pred[y_pred <= 0] = 1
y_pred[y_pred >= 2] = 2

In [26]:
# Model evaluation
accuracy = accuracy_score(y_test, y_pred.round())
precision = precision_score(y_test, y_pred.round(), average='weighted')
recall = recall_score(y_test, y_pred.round(), average='weighted')
f1 = f1_score(y_test, y_pred.round(), average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred.round())

In [27]:
print(accuracy, precision, recall, f1, conf_matrix)

0.926766192547314 0.9249312897098703 0.926766192547314 0.9194314890353225 [[7194   80]
 [ 543  690]]


# Fast Text 
### Linear Regression

## DF7

In [31]:
# Assuming 'df7' is your DataFrame with features and the target column
features = df7['content'].apply(lambda x: x.split(' ', 1)[1])  # Drop the target column to get the feature columns
target = df7['target'].apply(lambda x: int(x.split("__label__")[1]))  # Target column to predict

stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

features = features.apply(preprocess)

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    #X_train = X_train.apply(preprocess)
    #X_test = X_test.apply(preprocess)

    sentences = [sentence.split() for sentence in X_train]
    
    # Train the FastText model
    fasttext_model = FastText(sentences, vector_size=100, window=20, min_count=2, workers=4, seed=42)
    
    def vectorize(sentence):
        words = sentence.split()
        words_vecs = [fasttext_model.wv[word] for word in words if word in fasttext_model.wv]
        if len(words_vecs) == 0:
            return np.zeros(100)
        words_vecs = np.array(words_vecs)
        return words_vecs.mean(axis=0)

    X_train = np.array([vectorize(sentence) for sentence in X_train])
    X_test = np.array([vectorize(sentence) for sentence in X_test])
    
    #clf = LogisticRegression()
    clf = LinearRegression()
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    y_pred = np.round(y_pred)
    
    y_pred[y_pred <= 0] = 1
    y_pred[y_pred >= 2] = 2
    
    # Model evaluation
    accuracy = accuracy_score(y_test, y_pred.round())
    precision = precision_score(y_test, y_pred.round(), average='weighted')
    recall = recall_score(y_test, y_pred.round(), average='weighted')
    f1 = f1_score(y_test, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred.round())
    
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF7',
    'model' : 'Fast Text - Linear Regression',
    'data balancing technique' : 'None',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

Mean Accuracy: 0.93
Mean Precision: 0.92
Mean Recall: 0.93
Mean F1-Score: 0.92


In [32]:
combined_metrics[(combined_metrics['data balancing technique'] == 'None') & (combined_metrics['dataset'] == 'DF7')]#.head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
0,DF7,Fast Text - Linear Regression,None,1,0.927109,0.902572,0.989653,0.551954,0.957361,0.685004,12081.0,2098.0,11956,940,1158,125
1,DF7,Fast Text - Linear Regression,None,2,0.929642,0.890671,0.987557,0.575601,0.957725,0.699285,12055.0,2123.0,11905,901,1222,150
2,DF7,Fast Text - Linear Regression,None,3,0.925717,0.907847,0.989304,0.570330,0.956455,0.700556,11967.0,2211.0,11839,950,1261,128
3,DF7,Fast Text - Linear Regression,None,1,0.930376,0.906915,0.990393,0.558101,0.959447,0.690983,3643.0,611.0,3608,270,341,35
4,DF7,Fast Text - Linear Regression,None,2,0.931347,0.906091,0.989813,0.573955,0.959690,0.702756,3632.0,622.0,3595,265,357,37
5,DF7,Fast Text - Linear Regression,None,3,0.927848,0.902743,0.989209,0.565625,0.957547,0.695485,3614.0,640.0,3575,278,362,39
6,DF7,Fast Text - Linear Regression,None,4,0.927102,0.881356,0.986427,0.565217,0.955845,0.688742,3610.0,644.0,3561,280,364,49
7,DF7,Fast Text - Linear Regression,None,5,0.930081,0.885986,0.986715,0.581903,0.957561,0.702448,3613.0,641.0,3565,268,373,48
8,DF7,Fast Text - Linear Regression,None,6,0.933664,0.910377,0.989482,0.603125,0.960763,0.725564,3613.0,640.0,3575,254,386,38
9,DF7,Fast Text - Linear Regression,None,7,0.930836,0.908257,0.988867,0.600000,0.958974,0.722628,3593.0,660.0,3553,264,396,40


### FastText Linear Regression with df7 data and imbalance data tackling (RandomUnderSampler)

In [33]:
# Assuming 'df7' is your DataFrame with features and the target column
features = df7['content'].apply(lambda x: x.split(' ', 1)[1])  # Drop the target column to get the feature columns
target = df7['target'].apply(lambda x: int(x.split("__label__")[1]))  # Target column to predict

stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

features = features.apply(preprocess)

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    #X_train = X_train.apply(preprocess)
    #X_test = X_test.apply(preprocess)

    sentences = [sentence.split() for sentence in X_train]
    
    # Train the FastText model
    fasttext_model = FastText(sentences, vector_size=100, window=20, min_count=2, workers=4, seed=42)
    
    def vectorize(sentence):
        words = sentence.split()
        words_vecs = [fasttext_model.wv[word] for word in words if word in fasttext_model.wv]
        if len(words_vecs) == 0:
            return np.zeros(100)
        words_vecs = np.array(words_vecs)
        return words_vecs.mean(axis=0)

    X_train = np.array([vectorize(sentence) for sentence in X_train])
    X_test = np.array([vectorize(sentence) for sentence in X_test])
    
    # Apply random undersampling
    rus = RandomUnderSampler(random_state=42)
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
    
    #clf = LogisticRegression()
    clf = LinearRegression()
    clf.fit(X_train_resampled, y_train_resampled)
    
    # Apply random undersampling to test data
    X_test_resampled, y_test_resampled = rus.fit_resample(X_test, y_test)
    
    y_pred = clf.predict(X_test_resampled)
    
    y_pred = np.round(y_pred)
    
    y_pred[y_pred <= 0] = 1
    y_pred[y_pred >= 2] = 2
    
    # Model evaluation
    accuracy = accuracy_score(y_test_resampled, y_pred.round())
    precision = precision_score(y_test_resampled, y_pred.round(), average='weighted')
    recall = recall_score(y_test_resampled, y_pred.round(), average='weighted')
    f1 = f1_score(y_test_resampled, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test_resampled, y_pred.round())
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test_resampled, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test_resampled, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF7',
    'model' : 'Fast Text - Linear Regression',
    'data balancing technique' : 'Random Under Sampler',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

Mean Accuracy: 0.90
Mean Precision: 0.90
Mean Recall: 0.90
Mean F1-Score: 0.90


In [34]:
combined_metrics[(combined_metrics['data balancing technique'] == 'Random Under Sampler') & (combined_metrics['dataset'] == 'DF7')].head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
13,DF7,Fast Text - Linear Regression,Random Under Sampler,1,0.874003,0.894118,0.896890,0.870704,0.885299,0.882255,611.0,611.0,548,79,532,63
14,DF7,Fast Text - Linear Regression,Random Under Sampler,2,0.873016,0.882736,0.884244,0.871383,0.878594,0.877023,622.0,622.0,550,80,542,72


### FastText Linear Regression with df7 data and imbalance data tackling (RandomOverSampler)

In [37]:
# Assuming 'df7' is your DataFrame with features and the target column
features = df7['content'].apply(lambda x: x.split(' ', 1)[1])  # Drop the target column to get the feature columns
target = df7['target'].apply(lambda x: int(x.split("__label__")[1]))  # Target column to predict

stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

features = features.apply(preprocess)

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    #X_train = X_train.apply(preprocess)
    #X_test = X_test.apply(preprocess)

    sentences = [sentence.split() for sentence in X_train]
    
    # Train the FastText model
    fasttext_model = FastText(sentences, vector_size=100, window=20, min_count=2, workers=4, seed=42)
    
    def vectorize(sentence):
        words = sentence.split()
        words_vecs = [fasttext_model.wv[word] for word in words if word in fasttext_model.wv]
        if len(words_vecs) == 0:
            return np.zeros(100)
        words_vecs = np.array(words_vecs)
        return words_vecs.mean(axis=0)

    X_train = np.array([vectorize(sentence) for sentence in X_train])
    X_test = np.array([vectorize(sentence) for sentence in X_test])
    
    # Apply random undersampling
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
    
    #clf = LogisticRegression()
    clf = LinearRegression()
    clf.fit(X_train_resampled, y_train_resampled)
    
    # Apply random undersampling to test data
    X_test_resampled, y_test_resampled = rus.fit_resample(X_test, y_test)
    
    y_pred = clf.predict(X_test_resampled)
    
    y_pred = np.round(y_pred)
    
    y_pred[y_pred <= 0] = 1
    y_pred[y_pred >= 2] = 2
    
    # Model evaluation
    accuracy = accuracy_score(y_test_resampled, y_pred.round())
    precision = precision_score(y_test_resampled, y_pred.round(), average='weighted')
    recall = recall_score(y_test_resampled, y_pred.round(), average='weighted')
    f1 = f1_score(y_test_resampled, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test_resampled, y_pred.round())
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test_resampled, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test_resampled, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF7',
    'model' : 'Fast Text - Linear Regression',
    'data balancing technique' : 'Random Over Sampler',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

Mean Accuracy: 0.89
Mean Precision: 0.89
Mean Recall: 0.89
Mean F1-Score: 0.89


In [39]:
combined_metrics[(combined_metrics['data balancing technique'] == 'Random Over Sampler') & (combined_metrics['dataset'] == 'DF7')]#.head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
23,DF7,Fast Text - Linear Regression,Random Over Sampler,1,0.876404,0.891486,0.893617,0.873977,0.884927,0.882645,611.0,611.0,546,77,534,65
24,DF7,Fast Text - Linear Regression,Random Over Sampler,2,0.882911,0.895425,0.897106,0.881029,0.889952,0.888169,622.0,622.0,558,74,548,64
25,DF7,Fast Text - Linear Regression,Random Over Sampler,3,0.862385,0.878594,0.881250,0.859375,0.871716,0.868878,640.0,640.0,564,90,550,76
26,DF7,Fast Text - Linear Regression,Random Over Sampler,4,0.879499,0.873652,0.872671,0.880435,0.876072,0.877030,644.0,644.0,562,77,567,82
27,DF7,Fast Text - Linear Regression,Random Over Sampler,5,0.867580,0.886400,0.889236,0.864275,0.878274,0.875197,641.0,641.0,570,87,554,71
28,DF7,Fast Text - Linear Regression,Random Over Sampler,6,0.884127,0.872308,0.870313,0.885938,0.877165,0.879070,640.0,640.0,557,73,567,83
29,DF7,Fast Text - Linear Regression,Random Over Sampler,7,0.881818,0.881818,0.881818,0.881818,0.881818,0.881818,660.0,660.0,582,78,582,78
30,DF7,Fast Text - Linear Regression,Random Over Sampler,8,0.869310,0.881639,0.883582,0.867164,0.876388,0.874342,670.0,670.0,592,89,581,78
31,DF7,Fast Text - Linear Regression,Random Over Sampler,9,0.873303,0.874433,0.874622,0.873112,0.873962,0.873772,662.0,662.0,579,84,578,83
32,DF7,Fast Text - Linear Regression,Random Over Sampler,10,0.873860,0.892971,0.895639,0.870717,0.884615,0.881703,642.0,642.0,575,83,559,67


## Performance metrics for the Linear Regression

In [43]:
combined_metrics

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
0,DF7,Fast Text - Linear Regression,None,1,0.927109,0.902572,0.989653,0.551954,0.957361,0.685004,12081.0,2098.0,11956,940,1158,125
1,DF7,Fast Text - Linear Regression,None,2,0.929642,0.890671,0.987557,0.575601,0.957725,0.699285,12055.0,2123.0,11905,901,1222,150
2,DF7,Fast Text - Linear Regression,None,3,0.925717,0.907847,0.989304,0.570330,0.956455,0.700556,11967.0,2211.0,11839,950,1261,128
3,DF7,Fast Text - Linear Regression,None,1,0.930376,0.906915,0.990393,0.558101,0.959447,0.690983,3643.0,611.0,3608,270,341,35
4,DF7,Fast Text - Linear Regression,None,2,0.931347,0.906091,0.989813,0.573955,0.959690,0.702756,3632.0,622.0,3595,265,357,37
5,DF7,Fast Text - Linear Regression,None,3,0.927848,0.902743,0.989209,0.565625,0.957547,0.695485,3614.0,640.0,3575,278,362,39
6,DF7,Fast Text - Linear Regression,None,4,0.927102,0.881356,0.986427,0.565217,0.955845,0.688742,3610.0,644.0,3561,280,364,49
7,DF7,Fast Text - Linear Regression,None,5,0.930081,0.885986,0.986715,0.581903,0.957561,0.702448,3613.0,641.0,3565,268,373,48
8,DF7,Fast Text - Linear Regression,None,6,0.933664,0.910377,0.989482,0.603125,0.960763,0.725564,3613.0,640.0,3575,254,386,38
9,DF7,Fast Text - Linear Regression,None,7,0.930836,0.908257,0.988867,0.600000,0.958974,0.722628,3593.0,660.0,3553,264,396,40


In [44]:
# Assuming df is your DataFrame
combined_metrics.to_csv('Output Data/Fast Text - Linear Regression.csv')